(getting_started)=
# Getting Started


```{eval-rst}
We're going to demonstrate how to use `hbmep` to estimate recruitment curves from MEP size data using a hierarchical Bayesian model. The model is based on a rectified-logistic function (:cite:alps:`1987:nelson_`, see Methods 4.1.1, Eq. 4.1.4), which was introduced for parametric estimation of the threshold.
```


## A Simple Example


```{eval-rst}
.. note::

    This tutorial assumes that you are working on a Linux or MacOS system. If you are using Windows, you can use the Windows Subsystem for Linux (WSL), or update the bash commands and paths accordingly.

Begin by downloading the :file:`mock_data.csv` file from the `hbmep` repository using the following command.

.. code-block:: bash

    wget https://github.com/hbmep/hbmep/tree/main/docs/source/tutorials/mock_data.csv


This file contains transcranial magnetic stimulation (TMS) MEP size data for 3 participants, with responses from two muscles (PKPK_ECR and PKPK_FCR). The responses are recorded in peak-to-peak values in millivolts (mV), and the intensity is recorded in maximum stimulator output (0-100% MSO).
```

### Configuration file


```{eval-rst}
:file:`config.toml` is a configuration file used by `hbmep` that contains all the necessary information to run the model.
Begin by creating a TOML file, :file:`config.toml`, and enter the following content.

.. code-block:: toml

    [paths]
    csv_path = "./mock_data.csv"
    build_directory = "./mock_data_output/"

    [variables]
    intensity = "TMSIntensity"
    features = ["participant"]
    response = ['PKPK_ECR', 'PKPK_FCR']

    [mcmc]
    num_chains = 4
    num_warmup = 1000
    num_samples = 1000
    thinning = 1

    [misc]
    base = 10
```

```{eval-rst}
Alternatively, you can download this configuration file from the `hbmep` repository by running the following command:

.. code-block:: bash

    wget https://github.com/hbmep/hbmep/tree/main/docs/source/tutorials/config.toml

Here,

- ``paths`` table contains the paths to the dataset and the build directory.
    - ``csv`` is path to the dataset in csv format. In the TOML file, it points to the :file:`mock_data.csv` file.
    - ``build_dir`` is the directory where `hbmep` will store model artefacts, such as recruitment curve plots and estimates. If this directory does not already exists, `hbmep` will create it.
- ``variables`` table contains the names of the columns in the dataset that are used by the model.
    - ``intensity`` is the column name in the dataset that contains the TMS intensity.
    - ``features`` is a list of columns that uniquely identify the set of data points for a single recruitment curve. In this case, the model will yield recruitment curve for each participant.
    - ``response`` is a list of columns that contain the MEP size values. In this case, the model is run for two muscles, so the columns ``PKPK_ECR`` and ``PKPK_FCR`` are included in the list
- ``[mcmc]`` table configures the No-U-Turn sampler (NUTS) sampler.
    - ``num_chains`` is the number of chains to run in parallel.
    - ``num_warmup`` is the number of warmup samples.
    - ``num_samples`` is the number of samples to draw from the posterior.
    - ``thinning`` is the thinning factor for the samples.
```


### Running the model


```{eval-rst}
.. tip::

    If you have trouble running the commands in this tutorial, please copy the command
    and its output, then `open an issue <https://github.com/hbmep/hbmep/issues/new/choose>`_ on the `hbmep <https://github.com/hbmep/hbmep>`_ repository on
    GitHub. We'll do our best to help you!
```

In [13]:
import os
import pickle

import pandas as pd

from hbmep.config import Config
from hbmep.model.tms import RectifiedLogistic
from hbmep.model.utils import Site as site


Begin by reading the `mock_data.csv` file.

In [14]:
df = pd.read_csv("./mock_data.csv")

print(f"Dataframe shape:\t{df.shape}")
print(f"Dataframe columns:\t{', '.join(df.columns.tolist())}")
print(f"All participants:\t{', '.join(df['participant'].unique().tolist())}")

n = 5
print(f"\nDataframe first {n} rows:\n\n{df.head(n=n).to_string()}")


Dataframe shape:	(245, 4)
Dataframe columns:	TMSIntensity, participant, PKPK_ECR, PKPK_FCR
All participants:	P1, P2, P3

Dataframe first 5 rows:

   TMSIntensity participant  PKPK_ECR  PKPK_FCR
0         43.79          P1     0.197     0.048
1         55.00          P1     0.224     0.068
2         41.00          P1     0.112     0.110
3         43.00          P1     0.149     0.058
4         14.00          P1     0.014     0.011


(build-the-model)=
Now, we will build the model and process this dataset with it so that it's usable by _hbmep_.

In [15]:
# Build model
config = Config(toml_path="./config.toml")
model = RectifiedLogistic(config=config)

# Process data
df, encoder_dict = model.load(df=df)


Before running inference, we can visualize the dataset.


In [16]:
# Plot data
model.plot(df=df, encoder_dict=encoder_dict)


```{eval-rst}
The above command created a pdf :file:`dataset.pdf` and saved it to the build directory. It can also be accessed here :download:`dataset.pdf <mock_data_output/dataset.pdf>`. As it can be seen, there are three participants with their MEPs recorded from two muscles.
```


Now, lets run inference on this data.

In [17]:
# Run inference
mcmc, posterior_samples = model.run_inference(df=df)


  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

We can print the convergence diagnostics with the following command.


In [18]:
model.print_summary(samples=posterior_samples)



                 mean       std    median      2.5%     97.5%     n_eff     r_hat
     H[0,0]      0.24      0.01      0.24      0.21      0.27   2302.41      1.00
     H[0,1]      0.16      0.01      0.16      0.14      0.18   3441.27      1.00
     H[1,0]      0.19      0.04      0.18      0.14      0.26   1853.05      1.00
     H[1,1]      0.16      0.07      0.14      0.08      0.29   1617.89      1.00
     H[2,0]      0.23      0.05      0.22      0.15      0.34   3450.24      1.00
     H[2,1]      0.26      0.13      0.22      0.10      0.52   2259.82      1.00
    H_scale      0.28      0.13      0.25      0.12      0.54   1015.16      1.00
     L[0,0]      0.01      0.00      0.01      0.01      0.02   3529.07      1.00
     L[0,1]      0.01      0.00      0.01      0.01      0.01   3712.93      1.00
     L[1,0]      0.01      0.00      0.01      0.01      0.01   4051.42      1.00
     L[1,1]      0.01      0.00      0.01      0.01      0.01   3993.32      1.00
     L[2,0]    

```{eval-rst}
The different site names present in the first column of the above output correspond to model parameters (:cite:alps:`1987:nelson_`, see Methods 4.1.1, Supplementary Fig. ??)
```

### Plotting the curves

Now, we will plot the estimated recruitment curves. Before that, we need to use the estimated parameters and predict on a "template" prediction dataframe.

This prediction dataframe is similar to the input dataframe on which inference was run, except it has missing MEP size response values. Additionally, it has many more intensity values (controlled by the argument `num_points=100`). These are the intensity values on which the model will predict the response values.


In [19]:
# Create prediction dataframe
prediction_df = model.make_prediction_dataset(df=df, num_points=100)

# Use the model to predict on the prediction dataframe
posterior_predictive = model.predict(
    df=prediction_df, posterior_samples=posterior_samples
)


This returns the posterior predictive of the model. We can use this to plot the recruitment curves.


In [20]:
# Plot recruitment curves
model.render_recruitment_curves(
    df=df,
    encoder_dict=encoder_dict,
    posterior_samples=posterior_samples,
    prediction_df=prediction_df,
    posterior_predictive=posterior_predictive
)


```{eval-rst}
The above command created a pdf :file:`recruitment_curves.pdf` and saved it to the build directory. It can also be accessed here :download:`recruitment_curves.pdf <mock_data_output/recruitment_curves.pdf>`.

As it can be seen in the pdf, the participants are aligned top to bottom, and the muscles are aligned left to right. The plots are colored by muscle name. For a given participant and muscle combination, the first plot shows the MEP size and stimulation intensity data, the second plot shows the estimated recruitment curve overlaid on top of the data, and the third plot shows posterior distribution of the threshold parameter.

The estimated curves look good, except for participant P1 and muscle FCR, which is indexed by the tuple (0, 1). The curve seems to be biased towards a few outliers. Later on, we will see how to tackle this using a mixture model (:cite:alps:`1987:nelson_`, see Methods 4.1.1, Supplementary Fig. ??), but for now, we will go with it since the goal is to get comfortable with using `hbmep`.
```


Additionally, we can also plot the posterior predictive of the model using the following command.

In [21]:
# Posterior predictive check
model.render_predictive_check(
    df=df,
    encoder_dict=encoder_dict,
    prediction_df=prediction_df,
    posterior_predictive=posterior_predictive
)


```{eval-rst}
The above command created a pdf :file:`posterior_predictive_check.pdf` and saved it to the build directory. It can also be accessed here :download:`posterior_predictive_check.pdf <mock_data_output/posterior_predictive_check.pdf>`.
```

### Accessing parameters

The estimated parameters are stored in the `posterior_samples` dictionary. The keys are the parameter names and the values are the samples from the posterior distribution. The samples are stored as a NumPy array. The samples can be accessed as follows:


In [22]:
# Threshold parameter
a = posterior_samples[site.a]

print(f"Shape of a:\t{a.shape}")
print(f"\nFirst dimension corresponds to the number of samples:\t\t{a.shape[0]}")
print(f"Second dimension corresponds to the number of participants:\t{a.shape[1]}")
print(f"Third dimension corresponds to the number of muscles:\t\t{a.shape[2]}")


Shape of a:	(4000, 3, 2)

First dimension corresponds to the number of samples:		4000
Second dimension corresponds to the number of participants:	3
Third dimension corresponds to the number of muscles:		2


Note that there are 4000 samples because we ran 4 chains with 1000 samples each.

Additionally, we can reshape the array so that the first dimension corresponds to the chains and the second dimension corresponds to the samples. Although this is not necessary, it can be useful for some analyses.


In [23]:
# Reshape samples to group them by chain
a_grouped_by_chain = a.reshape(
    model.mcmc_params["num_chains"],
    -1,
    *a.shape[1:]
)

print(f"Shape of a_grouped_by_chain:\t{a_grouped_by_chain.shape}")


Shape of a_grouped_by_chain:	(4, 1000, 3, 2)


The other curve parameters can be accessed in a similar way.

<!-- However, there are two parameters which require a different kind of manipulation. Let's look at all the keys available and their shapes. -->


### Saving the model

In [24]:
# Save the model
destination_path = os.path.join(model.build_dir, "inference.pkl")

with open(destination_path, "wb") as f:
    pickle.dump((model, mcmc, posterior_samples,), f)

print(f"Model saved to:\t{destination_path}")


Model saved to:	./mock_data_output/inference.pkl


The saved model can be loaded and used for further analysis:

In [25]:
# Load the model
source_path = destination_path

with open(source_path, "rb") as f:
    model, mcmc, posterior_samples = pickle.load(f)


### Using other functions


```{eval-rst}
Alternatively, we can use other functions to estimate the recruitment curves. The available choices in `hbmep` (:cite:alps:`1987:nelson_`, see Results 2.3 and Fig. 3 for a comparison of their predictive performance to the rectified-logistic function, Methods 4.1, Eq. 4.1.1-4.1.3) are:
```

* Logistic-4, also known as the Boltzmann sigmoid.
* Logistic-5, which is a more generalized version of Logistic4.
* Rectified-linear.

We recommend using Logistic-5 if estimating threshold is not the goal.


For example, to use Logistic-5 function, we need to import it and modify the [code](build-the-model) appropriately.


In [26]:
# Import Logistic5 model
from hbmep.model.tms import Logistic5


# Build model
config = Config(toml_path="./config.toml")
model = Logistic5(config=config)


### References

```{eval-rst}
.. bibliography::
    :cited:
    :style: unsrt
```
